In [245]:
library(xml2)

In [246]:
ComXmlFile  = read_xml("data/com.xml")
ContratXmlFile = read_xml("data/contract.xml")
EngiXmlFile = read_xml("data/engi.xml")

In [247]:
# Chargement des bibliothèques nécessaires
library(xml2)     # Pour lire et manipuler des fichiers XML
library(dplyr)    # Pour manipuler des tableaux de données (tibbles)
library(purrr)    # Pour la fonctions de programmation fonctionnelle  (walk)
library(stringr)  # Pour manipuler les chaînes de caractères

# Fonction principale : transforme un document XML en une liste de tables par classe
xmlToTables <- function(doc){
  root <- xml_root(doc)  # Accède à la racine du document XML
  tables <- list()       # Initialise une liste vide pour stocker les tables

  # Fonction récursive interne pour analyser chaque objet XML
  parse_obj <- function(node){
    clas <- xml_attr(node, "clas")       # Récupère l’attribut "clas" (classe de l'objet)
    if (is.na(clas)) return()            # Ignore les nœuds sans classe

    # 1) Extraction des propriétés simples (texte uniquement, sans nœuds enfants)
    props <- xml_find_all(node, "./prop[not(*) and not(.//obj)]")
    row <- set_names(
      xml_text(props) %>% trimws(),                     # Valeurs texte nettoyées
      xml_attr(props, "nom") %>% str_remove("^\\$")     # Noms de colonnes, sans "$"
    )

    # Ajoute la ligne à la table correspondant à cette classe
    if (is.null(tables[[clas]])) tables[[clas]] <- tibble()
    tables[[clas]] <<- bind_rows(tables[[clas]], as_tibble_row(row))

    # Recherche des objets enfants dans <prop> ou <entr>
    children <- c(
      xml_find_all(node, "./prop//obj"),
      xml_find_all(node, "./prop//entr/obj")
    )

    # Appel récursif sur chaque enfant
    walk(children, parse_obj)
  }

  # Démarre l’analyse en profondeur sur tous les <obj> du document
  xml_find_all(root, ".//obj") %>% walk(parse_obj)

  # Nettoyage des noms de colonnes : suppression des caractères spéciaux
  tables <- map(tables, ~rename_with(., ~str_replace_all(.x, "[^[:alnum:]_]", "")))

  tables  # Retourne la liste des tables par classe
}


In [248]:
com_df  = xmlToTables(ComXmlFile)

In [249]:
com_df

$Comm
# A tibble: 1 × 98
  aNote documSource estCQ estCQSup estImport estReprise dateAcompte1Emis fgfExclu installExclu matExclu
  <chr> <chr>       <chr> <chr>    <chr>     <chr>      <chr>            <chr>    <chr>        <chr>   
1 false false       false false    false     false      2023/08/30       false    false        false   
# ℹ 88 more variables: moExclu <chr>, pourcAcompte1 <chr>, pourcAcompte2 <chr>, solde <chr>,
#   transportExclu <chr>, appNotifObligs <chr>, appNotifs <chr>, codeClient2 <chr>,
#   dateValidite <chr>, delaiPrevExped <chr>, disclaimers <chr>, emplacementCre <chr>, enErreur <chr>,
#   enPSD <chr>, identCour <chr>, nbJourValide <chr>, nomClient <chr>, projet <chr>, promotions <chr>,
#   qteItems <chr>, qteItemsCab <chr>, roleProprietaire <chr>, sorte <chr>, statutCJobsite <chr>,
#   statutCred <chr>, type <chr>, typeProcess <chr>, validCred <chr>, ligne <chr>,
#   avertLivraison <chr>, cleEnMain <chr>, codePost <chr>, installation <chr>, typeClient <chr>, …


In [250]:
contrat_df = xmlToTables(ContratXmlFile)
contrat_df

$Contrat
# A tibble: 1 × 86
  codeClient2 dateValidite delaiPrevExped emplacementCre identCour nbJourValide nomClient projet  
  <chr>       <chr>        <chr>          <chr>          <chr>     <chr>        <chr>     <chr>   
1 ALaplante   2025/03/10   2              Interne        0         60           ALaplante P-001666
# ℹ 78 more variables: promotions <chr>, qteItems <chr>, qteItemsCab <chr>, roleProprietaire <chr>,
#   sorte <chr>, type <chr>, typeClient <chr>, typeProcess <chr>, validCred <chr>, ligne <chr>,
#   cleEnMain <chr>, installateur <chr>, installation <chr>, priseMesure <chr>, assemblage <chr>,
#   edgeColle <chr>, gamme <chr>, cleItem <chr>, createurCourriel <chr>, createurNom <chr>,
#   dateCreation <chr>, division <chr>, documCree <chr>, sauvegarde <chr>, code <chr>, codeTaxe <chr>,
#   codeTaxe2 <chr>, pourcEsc <chr>, pourcEscSup <chr>, sousTotal <chr>, sousTotal2 <chr>,
#   tauxTaxe <chr>, tauxTaxe2 <chr>, total <chr>, totalHT <chr>, totalTx1 <chr>, totalTx2 <chr>

In [251]:
contrat_df$Disclaimer

# A tibble: 6 × 2
  condition terme          
  <chr>     <chr>          
1 ""        PriseMesure_100
2 ""        Electro_90     
3 ""        PriseMesure_100
4 ""        Electro_90     
5 ""        PriseMesure_100
6 ""        Electro_90     

In [252]:

conn = DBI::dbConnect(odbc::odbc(), .connection_string = "Driver={ODBC Driver 17 for SQL Server};", 
            Server = "server-sql-dunin-eve.database.windows.net", Database = "AviviaEve_Copy", 
            UID = "accesAviviaData", PWD = "Citric&Private&Designed&Maturity1&Cloning")




In [253]:
projets = tbl(conn,'projetEW') 

In [254]:
projets %>% colnames()

[1] "Code"         "Client"       "Createur"     "ProjetXML"    "Archive"      "DateCreation"
[7] "Libre"       

In [255]:
projet_df = projets %>% head(1) %>% pull(ProjetXML) %>% as_xml_document() %>% xmlToTables()

In [256]:
projet_df

$Projet
# A tibble: 1 × 22
  aDocuments client codeRepresentant codeRepresentant2 coordsFactTxt    enFusion estReprise libre sorte
  <chr>      <chr>  <chr>            <chr>             <chr>            <chr>    <chr>      <chr> <chr>
1 true       Avivia 003              004               "20 Route Goule… true     false      0     Proj…
# ℹ 13 more variables: ciClient <chr>, coordsExpedTxt <chr>, courrielContact <chr>,
#   dateCreation <chr>, division <chr>, memeCoordsJSCF <chr>, nomContact <chr>, nomRepresentant <chr>,
#   nomRepresentant2 <chr>, posteContact <chr>, sauvegarde <chr>, statutJobsite <chr>, code <chr>

$Client
# A tibble: 2 × 4
  langue monnaie code   nom   
  <chr>  <chr>   <chr>  <chr> 
1 fr     CAN     Avivia Avivia
2 fr     CAN     Avivia Avivia

$AdrCivique
# A tibble: 6 × 9
  codePost etatProv ligne1          pays   region ville                        usage code     sorte    
  <chr>    <chr>    <chr>           <chr>  <chr>  <chr>                        <chr> <chr>

In [258]:
projet_df$Projet

# A tibble: 1 × 22
  aDocuments client codeRepresentant codeRepresentant2 coordsFactTxt    enFusion estReprise libre sorte
  <chr>      <chr>  <chr>            <chr>             <chr>            <chr>    <chr>      <chr> <chr>
1 true       Avivia 003              004               "20 Route Goule… true     false      0     Proj…
# ℹ 13 more variables: ciClient <chr>, coordsExpedTxt <chr>, courrielContact <chr>,
#   dateCreation <chr>, division <chr>, memeCoordsJSCF <chr>, nomContact <chr>, nomRepresentant <chr>,
#   nomRepresentant2 <chr>, posteContact <chr>, sauvegarde <chr>, statutJobsite <chr>, code <chr>